In [17]:
# Importando as bibliotecas necessárias 
import mysql.connector
from mysql.connector import Error
import pandas as pd 
import csv 
import re


In [18]:
# Configuração da conexão
config = {
    'user': 'root',
    'password': 'Pingofofo01',
    'host': 'localhost',
    'database': 'otica_mais_visao',
    'raise_on_warnings': True
}
# Iniciando a conexão
try:
    conn = mysql.connector.connect(**config)
    print('Conexão bem-sucedida!')

except mysql.connector.Error as err:
    print(f'Erro ao conectar ao banco de dados: {err}')
finally:
    # Fechando a conexão
    if conn.is_connected():
        conn.close()
        print('Conexão fechada.')

Conexão bem-sucedida!
Conexão fechada.


In [19]:
# Excluindo as tabelas antes de cria-lás novamente 
try:
        conn = mysql.connector.connect(**config)
        delete_from_tabelas= conn.cursor()
        delete_from_tabelas.execute("SHOW TABLES")
        tables = delete_from_tabelas.fetchall()
# Trazendo todas as tabelas do banco
        for table in tables: 
            table_name = table[0]
            delete_query = f"DELETE FROM {table_name}"
            delete_from_tabelas.execute(delete_query)

        conn.commit()
        print("Registros excluídos com sucesso.")
except mysql.connector.Error as  err:
        print(f"Erro ao excluir registros: {err}")
finally: 
        if delete_from_tabelas:
            delete_from_tabelas.close()
        if  conn.is_connected():
             conn.close()
             print("Conexão fechada.")

Registros excluídos com sucesso.
Conexão fechada.


In [20]:
try:
    # Estabelecendo a conexão
    conn = mysql.connector.connect(**config)
    delete_tabelas = conn.cursor()
    # Desativando restrições de chave estrangeira
    delete_tabelas.execute("SET FOREIGN_KEY_CHECKS = 0")

    # Obter a lista de tabelas
    delete_tabelas.execute("SHOW TABLES")
    tables = delete_tabelas.fetchall()

    for table in tables: 
        table_name = table[0]
        drop_table_query = f"DROP TABLE {table_name}"
        delete_tabelas.execute(drop_table_query)
        print(f"Foi excluido com sucesso: {table_name}")

    # Ativando restrições de chave estrangeira novamente
    delete_tabelas.execute("SET FOREIGN_KEY_CHECKS = 1")

    conn.commit()

except mysql.connector.Error as  err:
        print(f"Erro ao excluir registros: {err}")

finally: 
        if delete_tabelas:
            delete_tabelas.close()
        if  conn.is_connected():
             conn.close()
             print("Conexão fechada.")


Foi excluido com sucesso: armacao
Foi excluido com sucesso: clientes
Foi excluido com sucesso: estoque
Foi excluido com sucesso: fluxo_caixa
Foi excluido com sucesso: fornecedores
Foi excluido com sucesso: funcionarios
Foi excluido com sucesso: lente
Foi excluido com sucesso: mercadoria
Foi excluido com sucesso: transacao
Foi excluido com sucesso: vendas
Conexão fechada.


In [21]:
# Criando a tabela
def criar_tabela():
    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()

        # Consultas SQL para criar a tabela
        CREATE_TABLE_ARMACAO = """
            create table armacao (
            id_produto int(5) not null primary key, 
            modelo varchar(255),
            marca varchar(255), 
            tamanho varchar(255) 
        );     
        """
        CREATE_TABLE_LENTE = """
            create table lente ( 
            id_produto int(5) not null primary key, 
            marca varchar(255),
            tamanho varchar(255),
            tipo varchar(255) 
        );
        """
        CREATE_TABLE_ESTOQUE = """
            create table estoque ( 
            codigo_barras int(12) not null primary key auto_increment, 
            quantidade int(10), 
            id_fornecedor int(10), 
            fornecedor varchar(255),
            descricao varchar(1000), 
            preco_unitario decimal(10) 
        );
        """
        CREATE_TABLE_MERCADORIA = """
            CREATE TABLE mercadoria (
            id_produto INT(5) NOT NULL AUTO_INCREMENT,
            nome_produto VARCHAR(255),
            descricao VARCHAR(1000),
            codigo_barras INT(12),
            valor_unitario DECIMAL(10, 2),
            qtd_recebida INT(10),
            armacao INT(5),
            lentes INT(5),
            estoque INT(5),
            PRIMARY KEY (id_produto),
            FOREIGN KEY (armacao) REFERENCES armacao(id_produto),
            FOREIGN KEY (estoque) REFERENCES estoque(codigo_barras)
        );
        """ 

        CREATE_TABLE_FORNECEDORES = """
            CREATE TABLE fornecedores (
            id_fornecedor INT(10) NOT NULL PRIMARY KEY AUTO_INCREMENT,
            telefone VARCHAR(11),
            razao_social VARCHAR(255),
            cnpj VARCHAR(14),
            endereco VARCHAR(255),
            id_produto INT(5) NOT NULL,
            descricao_produto VARCHAR(255),
            FOREIGN KEY (id_produto) REFERENCES mercadoria(id_produto)
        );
        """
        CREATE_TABLE_CLIENTE = """
            create table clientes ( 
            cpf varchar(11) not null primary key, 
            telefone varchar(11),
            nome varchar(255), 
            endereco varchar(255) 
        ); 
        """
        CREATE_TABLE_FUNCIONARIOS = """
            create table funcionarios ( 
            id_vendedor int(4) not null primary key auto_increment, 
            nome varchar(255),
            telefone varchar(11), 
            endereco varchar(255), 
            cpf varchar(11),
            valor_vendido decimal(15) 
        ); 
        """ 
        CREATE_TABLE_TRANSACAO = """
            CREATE TABLE transacao (
            id_produto INT(5) NOT NULL PRIMARY KEY AUTO_INCREMENT,
            id_venda INT(10) NOT NULL UNIQUE KEY,
            data_venda DATE,
            valor_unitario DECIMAL(10),
            quantidade INT(10),
            FOREIGN KEY (id_produto) REFERENCES mercadoria(id_produto)
         );
        """
        CREATE_TABLE_FLUXO_CAIXA = """
            create table fluxo_caixa ( 
            id_transacao int(12) not null primary key, 
            data_venda date,
            despesas decimal(10), 
            entradas decimal(10), 
            saldo_operacional decimal(10) 
        ); 
        """
        CREATE_TABLE_VENDAS = """
            CREATE TABLE vendas (
            id_venda INT(7) NOT NULL AUTO_INCREMENT PRIMARY KEY,
            valor_total DECIMAL(10),    
            data_venda DATE,
            id_vendedor INT(4),
            id_transacao INT(12),
            cpf_cliente varchar(11) not null,
            nome_cliente varchar(255),
            FOREIGN KEY (id_vendedor) REFERENCES funcionarios(id_vendedor),
            FOREIGN KEY (cpf_cliente) REFERENCES clientes(cpf),
            FOREIGN KEY (id_transacao) REFERENCES fluxo_caixa(id_transacao)
        );
        """   
    

     # Executando a consulta SQL
        cursor.execute(CREATE_TABLE_ARMACAO)
        cursor.execute(CREATE_TABLE_LENTE)
        cursor.execute(CREATE_TABLE_ESTOQUE)
        cursor.execute(CREATE_TABLE_MERCADORIA)
        cursor.execute(CREATE_TABLE_FORNECEDORES)
        cursor.execute(CREATE_TABLE_CLIENTE)
        cursor.execute(CREATE_TABLE_FUNCIONARIOS)
        cursor.execute(CREATE_TABLE_TRANSACAO)
        cursor.execute(CREATE_TABLE_FLUXO_CAIXA)
        cursor.execute(CREATE_TABLE_VENDAS)
        print("Tabelas criadas com sucesso")

    except mysql.connector.Error as err:
        print(f'Erro ao criar as tabelas: {err}')
        
    finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

criar_tabela()


Tabelas criadas com sucesso
Conexão fechada.


In [22]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('clientes.csv', 'r') as clientes_csv:
      leitor_csv = csv.reader(clientes_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_clientes = """
   INSERT INTO clientes (
      cpf
   ,  telefone
   ,  nome
   ,  endereco
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_clientes, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')


[['12345678911', '81991102340', 'Maria Alice', 'Rua Antonio Falcão'], ['24397810288', '79908235670', 'Ana Geovanna', 'Conselheiro Rosa e Silva'], ['72898717003', '71995678932', 'Luciana Batista', 'Rua dos Carijós'], ['25798052509', '61987603425', 'Igor Vilarim', 'Rua Domingos Olímpio'], ['90762891547', '85976345098', 'Juliana Siqueira', 'Rua Arlindo Nogueira'], ['22794093401', '91990507891', 'Lucas Gomes', 'Rua Carlos Augusto Cornelsen'], ['44806523917', '85998702389', 'Pedro Gustavo', 'Avenida Rio Branco'], ['17537840948', '81996789023', 'Edmilson Correia', 'Rodovia Raposo Tavares'], ['44294133812', '11997803450', 'Leticia Nunes', 'Avenida Governador José Malcher'], ['40120248020', '11998709870', 'Luisa Silva', 'Avenida Desembargador Moreira'], ['20430811145', '21998034787', 'Cassio Alves', 'Travessa da CDL'], ['11348170132', '71993704560', 'Victor Costa', 'Rua Frederico Moura'], ['20974822271', '51994502389', 'Maria Bravo', 'Rua Tenente-Coronel Cardoso'], ['40908539205', '51998304020

In [23]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('lentes.csv', 'r') as lentes_csv:
      leitor_csv = csv.reader(lentes_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_lentes = """
   INSERT INTO lente (
      id_produto
   ,  marca
   ,  tamanho
   ,  tipo
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_lentes, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['95550', ' HOYA', '30.25mm', 'Antirreflexo blue control'], ['45480', ' HOYA', '66.21mm', 'Antirreflexo Long life'], ['47749', ' HOYA', '42.46mm', 'Antirreflexo NO-risk'], ['94725', ' HOYA', '05.60mm', 'Antirreflexo CleanExtra'], ['68775', ' HOYA', '95.63mm', 'Antirreflexo Aqua'], ['65616', 'Carl Zeiss', '86.18mm', 'Antireflexo a zeiss'], ['42639', 'Carl Zeiss', '43.40mm', 'DuraVision Platinum'], ['51376', 'Carl Zeiss', '85.81mm', 'DuraVision Blue Protector'], ['98748', 'Carl Zeiss', '02.13mm', 'DuraVision Chrome'], ['63601', 'Carl Zeiss', '71.53mm', 'DuraVision silver'], ['44411', 'Carl Zeiss', '39.63mm', 'Zeiss DriveSafe'], ['18547', 'Lentes Essilor(Crisal e Varilux)', '38.13mm', ' lentes Crizal'], ['53618', 'Lentes Essilor(Crisal e Varilux)', '85.96mm', ' lentes Crizal prevencia'], ['69159', 'Lentes Essilor(Crisal e Varilux)', '40.21mm', ' lentes Crizal Sapphire HR'], ['33406', 'Lentes Essilor(Crisal e Varilux)', '50.70mm', ' lentes Crizal Rock'], ['10809', 'Lentes Essilor(Crisal e

In [24]:
# Inserindo dados nas tabelas através de arquivos csv
try:
   conn = mysql.connector.connect(**config)
   cursor = conn.cursor()

   with open('armacao.csv', 'r') as armacao_csv:
      leitor_csv = csv.reader(armacao_csv, delimiter=';')
      dados = list(leitor_csv)

   insert_armacao = """
   INSERT INTO armacao (
      id_produto
   ,  modelo
   ,  marca
   ,  tamanho
   ) 
   VALUES (%s, %s, %s, %s)
   """
   # Executar a inserção das linhas
   print(dados)
   cursor.executemany(insert_armacao, dados)
   # Confirmar as alterações
   conn.commit()
   # Fechar a conexão
   conn.close()

except mysql.connector.Error as err:
        print(f'Erro ao alimentar tabelas: {err}')
        
finally:
        # Fechando a conexão
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('Conexão fechada.')

[['11111', 'KP1112', 'KIPLING', '51'], ['22222', 'VO5043L', 'VOGUE', '54'], ['33333', 'CKJ18703', 'CK JEANS', '53'], ['44444', 'SK5452', 'SWAROVSKI', '52'], ['55555', 'AT7072', 'ATITUDE', '50'], ['66666', 'OOX8156', 'OAKLEY', '56'], ['77777', 'DK5000', 'DKNY', '51'], ['88888', '7130', 'NIKE', '54'], ['99999', 'BG7073I', 'BULGET', '52'], ['12121', 'CKJ20636', 'CK JEANS', '52'], ['13131', 'AT8093', 'ATITUDE', '60'], ['14141', 'RB3519L', 'RAYBAN', '63'], ['15151', 'L986', 'LACOSTE', '52'], ['16161', 'AN4251', 'ARNETTE', '58'], ['17171', 'DS68', 'EVOKE', '57']]
